# Brest Cancer Detection with PyTorch NN

## Imports

In [208]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler

## Load Data

In [209]:
X, y = load_breast_cancer(return_X_y=True)

### Split and Scale Data

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 

In [211]:
X_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]], shape=(455, 30))

In [212]:
y_train

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,

### Convert to Tensors

In [213]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()

y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)

## Model

In [227]:
class BCNet(nn.Module):
    def __init__(self):
        super(BCNet, self).__init__()
        self.fc1 = nn.Linear(30, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 16)
        self.bn3 = nn.BatchNorm1d(16)
        self.fc4 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = self.dropout(F.leaky_relu(self.bn1(self.fc1(x)), negative_slope=0.01))
        x = self.dropout(F.leaky_relu(self.bn2(self.fc2(x)), negative_slope=0.01))
        x = self.dropout(F.leaky_relu(self.bn3(self.fc3(x)), negative_slope=0.01))
        x = self.fc4(x)
        return x

### Training

In [228]:
k_folds = 5
epochs = 100
patience = 10

kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_tensor)):
    print(f"\n{'='*40}")
    print(f"FOLD {fold+1}/{k_folds}")
    print(f"{'='*40}")
    
    # Split data for this fold
    X_fold_train = X_train_tensor[train_idx]
    y_fold_train = y_train_tensor[train_idx]
    X_fold_val = X_train_tensor[val_idx]
    y_fold_val = y_train_tensor[val_idx]
    
    fold_loader = DataLoader(
        TensorDataset(X_fold_train, y_fold_train),
        batch_size=32, shuffle=True
    )
    
    # Fresh model for each fold
    model = BCNet()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        running_loss = 0.0
        for x_batch, y_batch in fold_loader:
            optimizer.zero_grad()
            loss = criterion(model(x_batch), y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(fold_loader)
        train_losses.append(epoch_loss)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(X_fold_val), y_fold_val).item()
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  Early stopping at epoch {epoch+1}")
                break
    
    # Restore best model and evaluate on fold validation set
    model.load_state_dict(best_model_state)
    model.eval()
    with torch.no_grad():
        val_preds = torch.sigmoid(model(X_fold_val))
        val_acc = ((val_preds >= 0.5) == y_fold_val).float().mean().item()
    
    fold_results.append({
        'fold': fold + 1,
        'val_acc': val_acc,
        'val_loss': best_val_loss,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'epochs_trained': len(train_losses)
    })
    
    print(f"  Best Val Loss: {best_val_loss:.4f} | Val Accuracy: {val_acc:.4f} | Epochs: {len(train_losses)}")

# Summary
print(f"\n{'='*40}")
print("K-FOLD SUMMARY")
print(f"{'='*40}")
accs = [r['val_acc'] for r in fold_results]
print(f"Fold Accuracies: {[f'{a:.4f}' for a in accs]}")
print(f"Mean Accuracy:   {sum(accs)/len(accs):.4f} ± {torch.tensor(accs).std().item():.4f}")


FOLD 1/5
  Early stopping at epoch 46
  Best Val Loss: 0.1012 | Val Accuracy: 0.9670 | Epochs: 46

FOLD 2/5
  Early stopping at epoch 47
  Best Val Loss: 0.0449 | Val Accuracy: 0.9890 | Epochs: 47

FOLD 3/5
  Early stopping at epoch 49
  Best Val Loss: 0.0510 | Val Accuracy: 0.9780 | Epochs: 49

FOLD 4/5
  Early stopping at epoch 47
  Best Val Loss: 0.0678 | Val Accuracy: 0.9890 | Epochs: 47

FOLD 5/5
  Early stopping at epoch 53
  Best Val Loss: 0.0550 | Val Accuracy: 0.9780 | Epochs: 53

K-FOLD SUMMARY
Fold Accuracies: ['0.9670', '0.9890', '0.9780', '0.9890', '0.9780']
Mean Accuracy:   0.9802 ± 0.0092


### Evaluation

In [229]:
# Retrain on full training data with best hyperparameters
final_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=32, shuffle=True
)

final_model = BCNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(final_model.parameters(), lr=0.001)

# Use the average epochs from K-Fold as a guide
avg_epochs = int(sum(r['epochs_trained'] for r in fold_results) / len(fold_results))
print(f"Training final model for {avg_epochs} epochs (avg from K-Fold)\n")

for epoch in range(avg_epochs):
    final_model.train()
    running_loss = 0.0
    for x_batch, y_batch in final_loader:
        optimizer.zero_grad()
        loss = criterion(final_model(x_batch), y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{avg_epochs}, Loss: {running_loss/len(final_loader):.4f}")

Training final model for 48 epochs (avg from K-Fold)

Epoch 10/48, Loss: 0.2824
Epoch 20/48, Loss: 0.1743
Epoch 30/48, Loss: 0.1460
Epoch 40/48, Loss: 0.1184


In [230]:
# Final evaluation on held-out test set
final_model.eval()
with torch.no_grad():
    test_logits = final_model(X_test_tensor)
    test_loss = criterion(test_logits, y_test_tensor).item()
    test_preds = torch.sigmoid(test_logits)
    test_acc = ((test_preds >= 0.5) == y_test_tensor).float().mean().item()

print(f"\nTest Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Test Loss:     0.0961
Test Accuracy: 0.9825
